Census of Drug and Alcohol Treatment Services in Northern Ireland:Breakdown by Service Type

In [19]:
from gssutils import *
if is_interactive():
    import requests
    from cachecontrol import CacheControl
    from cachecontrol.caches.file_cache import FileCache
    from cachecontrol.heuristics import LastModified
    from pathlib import Path

    session = CacheControl(requests.Session(),
                           cache=FileCache('.cache'),
                           heuristic=LastModified())

    sourceFolder = Path('in')
    sourceFolder.mkdir(exist_ok=True)

    inputURL = 'https://www.health-ni.gov.uk/sites/default/files/publications/dhssps/data-census-drug-alcohol-treatment-services.xlsx'
    inputFile = sourceFolder / 'data-census-drug-alcohol-treatment-services.xlsx'
    response = session.get(inputURL)
    with open(inputFile, 'wb') as f:
      f.write(response.content)
    tab = loadxlstabs(inputFile, sheetids='Table 3')[0]

Loading in/data-census-drug-alcohol-treatment-services.xlsx which has size 46265 bytes
Table names: ['Table 3']


In [20]:
observations = tab.excel_ref('B16').expand(DOWN).expand(RIGHT).is_not_blank() - tab.excel_ref('B22').expand(DOWN).expand(RIGHT)  


In [21]:
observations

{<G19 '*'>, <I16 '*'>, <M21 3.5>, <H16 '*'>, <K16 27.0>, <F21 '*'>, <F19 '*'>, <F20 88.8>, <D20 91.8>, <M18 211.0>, <F17 2203.0>, <L18 51.0>, <K18 45.0>, <F18 '*'>, <F16 '*'>, <B21 '*'>, <I17 4656.0>, <M16 417.0>, <J19 6.4>, <K21 2.2>, <K17 1964.0>, <J21 4.5>, <L21 3.8>, <E18 '*'>, <H21 '*'>, <L20 79.6>, <B20 98.9>, <G20 96.0>, <G21 '*'>, <C17 321.0>, <M20 89.5>, <B19 '-'>, <H20 76.3>, <H17 810.0>, <L19 16.6>, <B18 '*'>, <I18 '*'>, <C20 99.1>, <M17 5341.0>, <J20 89.1>, <J18 115.0>, <E20 96.1>, <I20 88.6>, <G16 '*'>, <M19 7.0>, <B17 94.0>, <L16 225.0>, <I21 '*'>, <B16 '-'>, <D17 270.0>, <K20 96.5>, <J16 165.0>, <E21 '*'>, <E17 685.0>, <E19 '-'>, <K19 1.3>, <L17 1080.0>, <G17 1643.0>, <H18 '*'>, <E16 '-'>, <H19 '*'>, <J17 2297.0>, <G18 '*'>, <I19 '*'>}

In [22]:
Service = tab.excel_ref('A15').expand(DOWN).is_not_blank()
Service

{<A21 'Mixed (%)'>, <A20 'Non-residential (%)'>, <A18 'Mixed'>, <A15 'Total'>, <A16 'Residential'>, <A19 'Residential (%)'>, <A17 'Non-residential '>}

In [23]:
Treatment = tab.excel_ref('B14').expand(RIGHT)
Treatment

{<D14 'Drugs & Alcohol'>, <I14 'Total'>, <H14 'Drugs & Alcohol'>, <K14 'Drugs Only'>, <M14 ''>, <L14 'Drugs & Alcohol'>, <J14 'Alcohol Only'>, <C14 'Drugs Only'>, <B14 'Alcohol Only'>, <F14 'Alcohol Only'>, <G14 'Drugs Only'>, <E14 'Total'>}

In [24]:
age = tab.excel_ref('B13').expand(RIGHT).is_not_blank()
age

{<J13 'Treatment Type'>, <F13 '18 and over'>, <B13 'Under 18 '>, <M13 'Overall Total'>}

In [25]:
Dimensions = [
            HDim(Treatment,'Treatment Type',DIRECTLY,ABOVE),
            HDim(Service,'Residential Status',DIRECTLY,LEFT),
            HDim(age,'Age',CLOSEST,LEFT),
            HDimConst('Measure Type', 'Count'),
            HDimConst('Unit','People'),
            HDimConst('Sex','Persons'),
            HDimConst('Period','1 March 2017')
            ]

In [26]:
c1 = ConversionSegment(observations, Dimensions, processTIMEUNIT=True)
# savepreviewhtml(c1)

In [27]:
new_table = c1.topandas()
new_table.loc[new_table['Age'] == 'Treatment Type', 'Age'] = 'All Ages'
new_table.loc[new_table['Age'] == 'Overall Total', 'Age'] = 'All Ages'
new_table.loc[new_table['Treatment Type'] == '', 'Treatment Type'] = 'Total'
new_table

,OBS,DATAMARKER,Treatment Type,Residential Status,Age,Measure Type,Unit,Sex,Period
0,,-,Alcohol Only,Residential,Under 18,Count,People,Persons,1 March 2017
1,,-,Total,Residential,Under 18,Count,People,Persons,1 March 2017
2,,*,Alcohol Only,Residential,18 and over,Count,People,Persons,1 March 2017
3,,*,Drugs Only,Residential,18 and over,Count,People,Persons,1 March 2017
4,,*,Drugs & Alcohol,Residential,18 and over,Count,People,Persons,1 March 2017
5,,*,Total,Residential,18 and over,Count,People,Persons,1 March 2017
6,165,NaN,Alcohol Only,Residential,All Ages,Count,People,Persons,1 March 2017
7,27,NaN,Drugs Only,Residential,All Ages,Count,People,Persons,1 March 2017
8,225,NaN,Drugs & Alcohol,Residential,All Ages,Count,People,Persons,1 March 2017
9,417,NaN,Total,Residential,All Ages,Count,People,Persons,1 March 2017


In [10]:
new_table.columns = ['Value' if x=='OBS' else x for x in new_table.columns]

In [11]:
new_table.dtypes

Value                 object
DATAMARKER            object
Treatment Type        object
Residential Status    object
Age                   object
Measure Type          object
Unit                  object
Sex                   object
Period                object
dtype: object

In [12]:
new_table.tail(5)

,Value,DATAMARKER,Treatment Type,Residential Status,Age,Measure Type,Unit,Sex,Period
71,,*,Total,Mixed (%),18 and over,Count,People,Persons,1 March 2017
72,4.5,NaN,Alcohol Only,Mixed (%),Treatment Type,Count,People,Persons,1 March 2017
73,2.2,NaN,Drugs Only,Mixed (%),Treatment Type,Count,People,Persons,1 March 2017
74,3.8,NaN,Drugs & Alcohol,Mixed (%),Treatment Type,Count,People,Persons,1 March 2017
75,3.5,NaN,None,Mixed (%),Overall Total,Count,People,Persons,1 March 2017


In [13]:
new_table.count()

Value                 76
DATAMARKER            24
Treatment Type        69
Residential Status    76
Age                   76
Measure Type          76
Unit                  76
Sex                   76
Period                76
dtype: int64

In [14]:
new_table = new_table[new_table['Value'] !=  0 ]

In [15]:
new_table = new_table[new_table['Value'] !=  '' ]

In [16]:
new_table.count()

Value                 52
DATAMARKER             0
Treatment Type        45
Residential Status    52
Age                   52
Measure Type          52
Unit                  52
Sex                   52
Period                52
dtype: int64

In [17]:
def user_perc(x):
    
    if str(x) == 'Treatment Type':
        return 'All years'
    else:
        return x
    
new_table['Age'] = new_table.apply(lambda row: user_perc(row['Age']), axis = 1)

In [18]:
new_table['Treatment Type'].fillna('Total', inplace = True)

In [19]:
new_table['Service Type'] = 'All'
# new_table['Residential Status'] = 'All'
new_table['Health and Social Care Trust']  = 'All'

In [20]:
new_table = new_table[['Period', 'Sex', 'Age', 'Service Type', 'Residential Status', 'Treatment Type', 'Health and Social Care Trust', 'Measure Type', 'Unit', 'Value']]

In [21]:
new_table.head(5)

,Period,Sex,Age,Service Type,Residential Status,Treatment Type,Health and Social Care Trust,Measure Type,Unit,Value
0,1 March 2017,Persons,Under 18,All,Total,Alcohol Only,All,Count,People,95
1,1 March 2017,Persons,Under 18,All,Total,Drugs Only,All,Count,People,324
2,1 March 2017,Persons,Under 18,All,Total,Drugs & Alcohol,All,Count,People,294
3,1 March 2017,Persons,Under 18,All,Total,Total,All,Count,People,713
4,1 March 2017,Persons,18 and over,All,Total,Alcohol Only,All,Count,People,2482
